In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pandas as pd
import re
import json
import h2o

from datetime import datetime

from utils import get_data, Config, clean_url, make_clickable, add_columns

from tree import make_tree

In [16]:
# Load all data (except unknown/manual testing)
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Browser".name != 'Unknown';
"""
df = get_data(Config(), initial_data)
df = add_columns(df)

Connecting to the PostgreSQL database...
Connection successful


## Overview

In [17]:
# In total 3 runs + couple of "repeat"-mode runs
# Lot's of failures in xvfb mode!
df.groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"])["id"].count().sort_values(ascending=False).to_frame().reset_index()

,automation_mode,browser_id,name,version,os,headless_mode,id
0,selenium,14,firefox,119,Ubuntu 22.04,headless,506810
1,selenium,13,chrome,119,Ubuntu 22.04,headless-new,506394
2,selenium,15,edge,119,Ubuntu 22.04,headless-new,506394
3,selenium,17,brave,1.60.114 (119.0.6045.124),Ubuntu 22.04,headless-new,506357
4,selenium,20,edge,119,Ubuntu 22.04,xvfb,438937
5,selenium,19,firefox,119,Ubuntu 22.04,xvfb,412951
6,selenium,18,chrome,119,Ubuntu 22.04,xvfb,403415
7,selenium,22,brave,1.60.114 (119.0.6045.124),Ubuntu 22.04,xvfb,334811


## Error analysis

In [18]:
# Timeout (2), Fail (1), Not-run (3)
filtered_df = df.groupby(["browser"])["test_status"].filter(lambda x: x.nunique() >= 2)
display(df[["browser", "test_status"]].loc[df.index.isin(filtered_df.index)].groupby(["browser"])["test_status"].value_counts().to_frame())

df.groupby("test_status")["browser"].value_counts().to_frame()

count
browser                                            test_status        
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) se... 0            506068
                                                   1               288
                                                   2                 1
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) se... 0            334679
                                                   1               132
chrome Ubuntu 22.04 119 selenium headless-new      0            506105
                                                   1               288
                                                   2                 1
chrome Ubuntu 22.04 119 selenium xvfb              0            403225
                                                   1               188
                                                   2                 2
edge Ubuntu 22.04 119 selenium headless-new        0            506104
                                                   1               288
                                                   2                 2
edge Ubuntu 22.04 119 selenium xvfb                0            438669
                                                   1               264
                                                   2                 4
firefox Ubuntu 22.04 119 selenium headless         0            505662
                                                   2               860
                                                   3               180
                                                   1               108
firefox Ubuntu 22.04 119 selenium xvfb             0            411975
                                                   2               760
                                                   3               144
                                                   1                72

count
test_status browser                                                   
0           chrome Ubuntu 22.04 119 selenium headless-new       506105
            edge Ubuntu 22.04 119 selenium headless-new         506104
            brave Ubuntu 22.04 1.60.114 (119.0.6045.124) se...  506068
            firefox Ubuntu 22.04 119 selenium headless          505662
            edge Ubuntu 22.04 119 selenium xvfb                 438669
            firefox Ubuntu 22.04 119 selenium xvfb              411975
            chrome Ubuntu 22.04 119 selenium xvfb               403225
            brave Ubuntu 22.04 1.60.114 (119.0.6045.124) se...  334679
1           brave Ubuntu 22.04 1.60.114 (119.0.6045.124) se...     288
            chrome Ubuntu 22.04 119 selenium headless-new          288
            edge Ubuntu 22.04 119 selenium headless-new            288
            edge Ubuntu 22.04 119 selenium xvfb                    264
            chrome Ubuntu 22.04 119 selenium xvfb                  188
            brave Ubuntu 22.04 1.60.114 (119.0.6045.124) se...     132
            firefox Ubuntu 22.04 119 selenium headless             108
            firefox Ubuntu 22.04 119 selenium xvfb                  72
2           firefox Ubuntu 22.04 119 selenium headless             860
            firefox Ubuntu 22.04 119 selenium xvfb                 760
            edge Ubuntu 22.04 119 selenium xvfb                      4
            edge Ubuntu 22.04 119 selenium headless-new              2
            chrome Ubuntu 22.04 119 selenium xvfb                    2
            brave Ubuntu 22.04 1.60.114 (119.0.6045.124) se...       1
            chrome Ubuntu 22.04 119 selenium headless-new            1
3           firefox Ubuntu 22.04 119 selenium headless             180
            firefox Ubuntu 22.04 119 selenium xvfb                 144

In [19]:
# Teststatus == 1 (fail)

# All browsers have it
display(df[df["test_status"] == 1]["browser"].value_counts().to_frame())

# Only for upgradeHSTS_direct and upgradeHSTS_subdomain, they always result in test status 1 in the same browser!
display(df.loc[df["full_url"].isin(df.loc[df["test_status"] == 1]["full_url"])].groupby(["test_name", "response_id"])["test_status"].value_counts().to_frame().head())

# The responses that cause it all have \x00 in them!
offending_headers = {}
visited_sets = set()
for group, gdf in df.loc[df["test_status"] == 1].groupby("browser"):
    unique_headers_set = frozenset(gdf["raw_header"].apply(bytes).apply(lambda x: json.loads(x.decode("utf-8"))).astype(str).unique())
    if unique_headers_set not in visited_sets:
        visited_sets.add(unique_headers_set)
        offending_headers[unique_headers_set] = set([group])
    else:
        offending_headers[unique_headers_set].add(group)
for key, value in offending_headers.items():
    print(f"{value}:\n{key}\n\n")

# The offending URLs (without timeout is from the repeat run)
# Firefox has none without timeout as these tests failed for all browsers, thus create_repeat.py did not create repeat runs for Firefox!
offending_urls = {}
visited_sets = set()

for group, gdf in df.loc[df["test_status"] == 1].groupby("browser"):
    unique_urls_set = frozenset(gdf["clean_url"].unique())
    print(group, len(unique_urls_set))

    if unique_urls_set not in visited_sets:
        visited_sets.add(unique_urls_set)
        offending_urls[unique_urls_set] = set([group])
    else:
        offending_urls[unique_urls_set].add(group)
for key, value in offending_urls.items():
    print(f"{value}:\n{key}\n\n")

# 9 URLs failed in all browsers, 9 only failed in Chromium-based browsers (~~18x in clean_url as we have one for repeat and one without~~ we now remove timeout in clean_url)
# Firefox only fails if \x00 is in the value, if it is in the header names it does not care?!
# (manual test: Safari seems to fail for HTTP only, does not fail if the response is received via HTTPS?, in addition Safari also only cares about \x00 in values and not in header names)

,count
browser,
chrome Ubuntu 22.04 119 selenium headless-new,288
edge Ubuntu 22.04 119 selenium headless-new,288
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,288
edge Ubuntu 22.04 119 selenium xvfb,264
chrome Ubuntu 22.04 119 selenium xvfb,188
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,132
firefox Ubuntu 22.04 119 selenium headless,108
firefox Ubuntu 22.04 119 selenium xvfb,72


count
test_name          response_id test_status       
upgradeHSTS_direct 29713       1               52
                   29745       1               52
                   29816       1               54
                   30133       1               52
                   30165       1               52

{'chrome Ubuntu 22.04 119 selenium headless-new', 'brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb', 'edge Ubuntu 22.04 119 selenium xvfb', 'edge Ubuntu 22.04 119 selenium headless-new', 'brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new'}:
frozenset({"[['strict-transport-security', '\\x00max-age=60']]", "[['strict-transport-security', 'max-age=60\\x00']]", "[['strict-transport-security', 'max-age=0\\x00']]", "[['\\x00strict-transport-security', 'max-age=60']]", "[['strict-transport-security\\x00', 'max-age=60']]", "[['\\x00strict-transport-security', 'max-age=0']]", "[['strict-transport-security', '\\x00max-age=0']]", "[['strict-transport-security', 'max-\\x00age=0']]", "[['strict-trans\\x00port-security', 'max-age=20; includeSubDomains']]", "[['strict-transport-security', 'max-a\\x00ge=60']]", "[['strict-trans\\x00port-security', 'max-age=60']]", "[['strict-transport-security\\x00', 'max-age=0']]", "[['strict-trans\\x00port-security', 'max-age=0']]", "[['\

In [20]:
# Teststatus == 2 (timeout)

# Firefox a lot, others rarely
display(df[df["test_status"] == 2]["browser"].value_counts().to_frame())

# fetch_GET|TEST always timeout, others only sometimes (accidental timeout in other browsers!), upgradeHSTS (see below with teststatus==3)
display(df.loc[df["full_url"].isin(df.loc[df["test_status"] == 2]["full_url"])].groupby(["test_name"])["test_status"].value_counts().to_frame())

# The responses (for fetch_GET|TEST) that cause it all have \n in them?
display(df.loc[(df["test_status"] == 2) & (df["test_name"].isin(["fetch_GET", "fetch_TEST"]))]["raw_header"].apply(bytes).apply(lambda x: json.loads(x.decode("utf-8"))).astype(str).unique())

# The offending URLs (without timeout is from the repeat run)
display(df.loc[(df["test_status"] == 2) & (df["test_name"].isin(["fetch_GET", "fetch_TEST"]))]["clean_url"].unique())


# Firefox times out if \n in header?
# (Safari only times out for some of the tests, for the others fech fails. Chrome always fetch fails?!)

,count
browser,
firefox Ubuntu 22.04 119 selenium headless,860
firefox Ubuntu 22.04 119 selenium xvfb,760
edge Ubuntu 22.04 119 selenium xvfb,4
edge Ubuntu 22.04 119 selenium headless-new,2
chrome Ubuntu 22.04 119 selenium xvfb,2
chrome Ubuntu 22.04 119 selenium headless-new,1
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,1


count
test_name             test_status       
fetch_GET             2             1134
fetch_TEST            2              378
framing_iframe        2                2
                      0                2
fullscreen_iframe     0               23
                      2                5
oac_window.open       0                2
                      2                1
perfAPI_img           0                2
                      2                2
upgradeHSTS_direct    2              108
                      3              108
upgradeHSTS_subdomain 3              216

array(["[['access-control-allow-credentials', 'true\\n'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-methods', 'TEST'], ['access-control-allow-headers', 'Test'], ['access-control-expose-headers', 'Test']]",
       "[['\\naccess-control-allow-origin', '*'], ['Test', 'Test'], ['access-control-allow-credentials', 'true'], ['access-control-allow-methods', 'TEST'], ['access-control-allow-headers', 'Test'], ['access-control-expose-headers', 'Test']]",
       "[['access-control-allow-origin', 'https://sub.headers.websec.saarland\\n'], ['Test', 'Test'], ['access-control-allow-credentials', 'true'], ['access-control-allow-methods', 'TEST'], ['access-control-allow-headers', 'Test'], ['access-control-expose-headers', 'Test']]",
       "[['access-control-allow-headers', '*\\n'], ['Test', 'Test'], ['access-control-allow-origin', 'https://sub.headers.websec.saarland'], ['access-control-allow-credentials', 'true'], ['access-control

array(['http://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?resp_type=parsing&browser_id=1&label=CORS-ACAC&first_id=10759&last_id=10759&scheme=http',
       'https://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?resp_type=parsing&browser_id=1&label=CORS-ACAC&first_id=10759&last_id=10759&scheme=https',
       'http://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?resp_type=parsing&browser_id=1&label=CORS-ACAO&first_id=8028&last_id=8028&scheme=http',
       'http://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?resp_type=parsing&browser_id=1&label=CORS-ACAO&first_id=9689&last_id=9689&scheme=http',
       'http://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?resp_type=parsing&browser_id=1&label=CORS-ACAH&first_id=13505&last_id=13505&scheme=http',
       'http://sub.headers.websec.saarland/_hp/tests/fetch-cors.sub.html?resp_type=parsing&browser_id=1&label=CORS-ACEH&first_id=14593&last_id=14593&scheme=http',
       'https://sub.head

In [21]:
# Teststatus == 3 (not run)

# Only Firefox
display(df[df["test_status"] == 3]["browser"].value_counts().to_frame())

# Only for upgradeHSTS_direct and upgradeHSTS_subdomain, the later always is not run, the former is sometimes 2 (timeout)
display(df.loc[df["full_url"].isin(df.loc[df["test_status"] == 3]["full_url"])].groupby(["test_name", "response_id"])["test_status"].value_counts().to_frame())

# The responses that cause it all have \n in them?
display(df.loc[df["test_status"] == 3]["raw_header"].apply(bytes).apply(lambda x: json.loads(x.decode("utf-8"))).astype(str).unique())

# The offending URLs (without timeout is from the repeat run)
display(df.loc[df["test_status"] == 3]["clean_url"].unique())

# Reason if there is a newline in the response the fetch call will timeout in Firefox.
# As we implemented the testcases as promise tests, only the first test executes and all the others will always timeout.
# Chromium-based browsers do not time out and instead parse the fetch until the newline only and stop there. (same for safari, safari has loading icon in devtools however)

,count
browser,
firefox Ubuntu 22.04 119 selenium headless,180
firefox Ubuntu 22.04 119 selenium xvfb,144


count
test_name             response_id test_status       
upgradeHSTS_direct    29885       3               18
                                  2               18
                      30305       2               18
                                  3               18
                      30725       3               18
                                  2               18
                      31838       3               18
                                  2               18
                      32324       3               18
                                  2               18
                      32695       2               18
                                  3               18
upgradeHSTS_subdomain 29885       3               36
                      30305       3               36
                      30725       3               36
                      31838       3               36
                      32324       3               36
                      32695       3               36

array(["[['\\nstrict-transport-security', 'max-age=60']]",
       "[['\\nstrict-transport-security', 'max-age=0']]",
       "[['\\nstrict-transport-security', 'max-age=20; includeSubDomains']]",
       "[['strict-transport-security', 'max-age=0\\n']]",
       "[['strict-transport-security', 'max-age=20; includeSubDomains\\n']]",
       "[['strict-transport-security', 'max-age=60\\n']]"], dtype=object)

array(['http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=30305&last_id=30305&scheme=http',
       'http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=29885&last_id=29885&scheme=http',
       'http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=30725&last_id=30725&scheme=http',
       'http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=31838&last_id=31838&scheme=http',
       'http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=32695&last_id=32695&scheme=http',
       'http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=32324&last_id=32324&scheme=http'],
      dtype=object)

## General info/Test statistics

In [31]:
df[["browser", "test_name", "relation_info", "response_id", "org_scheme", "org_host", "resp_scheme", "resp_host"]].drop_duplicates().groupby("browser").count()

,test_name,relation_info,response_id,org_scheme,org_host,resp_scheme,resp_host
browser,,,,,,,
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,168774,168774,168774,168774,168774,168774,168774
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,168774,168774,168774,168774,168774,168774,168774
chrome Ubuntu 22.04 119 selenium headless-new,168774,168774,168774,168774,168774,168774,168774
chrome Ubuntu 22.04 119 selenium xvfb,168770,168770,168770,168770,168770,168770,168770
edge Ubuntu 22.04 119 selenium headless-new,168774,168774,168774,168774,168774,168774,168774
edge Ubuntu 22.04 119 selenium xvfb,168774,168774,168774,168774,168774,168774,168774
firefox Ubuntu 22.04 119 selenium headless,168768,168756,168768,168768,168768,168756,168756
firefox Ubuntu 22.04 119 selenium xvfb,168768,168756,168768,168768,168768,168756,168756


In [22]:
df.groupby("browser")["clean_url"].nunique()

browser
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new    85128
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb            85128
chrome Ubuntu 22.04 119 selenium headless-new                         85128
chrome Ubuntu 22.04 119 selenium xvfb                                 85127
edge Ubuntu 22.04 119 selenium headless-new                           85128
edge Ubuntu 22.04 119 selenium xvfb                                   85128
firefox Ubuntu 22.04 119 selenium headless                            85128
firefox Ubuntu 22.04 119 selenium xvfb                                85128
Name: clean_url, dtype: int64

In [23]:
df.groupby("browser")["response_id"].nunique()

browser
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new    44226
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb            44226
chrome Ubuntu 22.04 119 selenium headless-new                         44226
chrome Ubuntu 22.04 119 selenium xvfb                                 44225
edge Ubuntu 22.04 119 selenium headless-new                           44226
edge Ubuntu 22.04 119 selenium xvfb                                   44226
firefox Ubuntu 22.04 119 selenium headless                            44226
firefox Ubuntu 22.04 119 selenium xvfb                                44226
Name: response_id, dtype: int64

In [25]:
df.groupby(["test_name", "relation_info"])["response_id"].nunique()

test_name                   relation_info 
accesswindow_direct         direct            2656
fetch_GET                   credentials       8059
                            custom_headers    8059
                            simple            8059
fetch_TEST                  custom_method     8059
framing_iframe              direct            8780
                            nested            8780
fullscreen_iframe           child_allow       3880
                            direct            3880
imgloading_iframe           direct            2710
oac_window.open             window.open       1699
perfAPI_img                 direct            1601
referrer_iframe             iframe            2797
script_execution_iframe     direct            2726
subresourceloadingCOEP_img  direct            2637
subresourceloadingCORP_img  direct            3357
upgradeHSTS_direct          direct            3324
upgradeHSTS_subdomain       subdomain         3324
Name: response_id, dtype: int64

## Stability

- Apart from a handfull of timeouts (~10/1M?) only a single test with different outcome in the three test repetitions! Very stable!

In [34]:
df_org = df
df = df.loc[df["test_status"] == 0]

In [35]:
for _, os_group in df.groupby("os"):
    stab = os_group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])[["outcome_str"]].agg(["count", "nunique"])
    # Tests with more than one outcome
    diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
    print(f"{len(diff_outcomes)} tests have different outcomes!")
    display(diff_outcomes.reset_index()["browser"].value_counts().to_frame())
    display(diff_outcomes.reset_index()[["test_name", "browser"]].value_counts().to_frame())

    diff_with_responses = diff_outcomes.reset_index().groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "browser"])["response_id"].agg(list).to_frame().reset_index()


    for id, (test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, browser, response_ids) in diff_with_responses.iterrows():
        # TODO: shortcut, only show th first result for each test_name/browser; results are usually always the same!
        print(test_name, response_ids)
        rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"].isin(response_ids)) & (df["browser"] == browser)]
        with pd.option_context("display.max_colwidth", 200):
            display(rows[["outcome_str", "name"]].value_counts().to_frame())
            disp = rows.drop_duplicates(subset="outcome_str")[["browser", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
            disp = disp.style.format({'full_url': make_clickable})
            display(disp)
        # input("Continue!")

1 tests have different outcomes!


,count
browser,
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,1


,,count
"(test_name, )","(browser, )",
accesswindow_direct,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,1


accesswindow_direct [23242]


,,count
outcome_str,name,
"{'window.open.opener': 'object ""[object Window]""'}",brave,2
{'window.open.opener': 'null'},brave,1


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
2721199,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,"{'window.open.opener': 'object ""[object Window]""'}",,accesswindow_direct,https,sub.headers.websec.saarland,https,direct,headers.webappsec.eu,https://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?resp_type=parsing&browser_id=1&label=COOP&first_id=23242&last_id=23242&scheme=https
3563981,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,{'window.open.opener': 'null'},,accesswindow_direct,https,sub.headers.websec.saarland,https,direct,headers.webappsec.eu,https://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?resp_type=parsing&browser_id=1&label=COOP&first_id=23242&last_id=23242&scheme=https


## Xvfb vs headless?! 
- Simply use the same code as for stability but use `name` (browser_name) instead of `browser`
- This currently works as we only have one os and version per desktop browser!

- Almost no differences! (total of 23/(168774*4))
- Some are due to timeouts in Xvfb (e.g., all of fullscreen_iframe and framing_iframe seem to be timeout in Xvfb) -> headless only is better?
- One systematic difference? accesswindow_direct (COOP) if the response triggers a download Brave headfull/Xvfb records a window reference (as long as the download popup is not closed), whereas headless record null; all other browsers records null as well? -> thus only testing headless should be okay? (responses with \n in headers trigger a download)

In [42]:
for _, os_group in df.groupby("os"):
    stab = os_group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "name"])[["outcome_str"]].agg(["count", "nunique"])
    # Tests with more than one outcome
    diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
    print(f"{len(diff_outcomes)} tests have different outcomes!")
    display(diff_outcomes.reset_index()["name"].value_counts().to_frame())
    display(diff_outcomes.reset_index()[["test_name", "name"]].value_counts().to_frame())

    diff_with_responses = diff_outcomes.reset_index().groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "name"])["response_id"].agg(list).to_frame().reset_index()


    for id, (test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, browser, response_ids) in diff_with_responses.iterrows():
        # TODO: shortcut, only show th first result for each test_name/browser; results are usually always the same!
        print(test_name, response_ids)
        rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"].isin(response_ids)) & (df["name"] == browser)]
        with pd.option_context("display.max_colwidth", 200):
            display(rows[["outcome_str", "browser"]].value_counts().to_frame())
            disp = rows.drop_duplicates(subset="outcome_str")[["browser", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
            disp = disp.style.format({'full_url': make_clickable})
            display(disp)
        # input("Continue!")

23 tests have different outcomes!


,count
name,
chrome,12
brave,11


,,count
"(test_name, )","(name, )",
accesswindow_direct,brave,11
fullscreen_iframe,chrome,10
framing_iframe,chrome,2


accesswindow_direct [21640, 22059, 22478, 23242, 23606, 23990]


,,count
outcome_str,browser,
{'window.open.opener': 'null'},brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,18
"{'window.open.opener': 'object ""[object Window]""'}",brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,9


...

,,count
outcome_str,browser,
fullscreenEnabled: true,chrome Ubuntu 22.04 119 selenium headless-new,15
message timeout,chrome Ubuntu 22.04 119 selenium xvfb,5


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
1822472,chrome Ubuntu 22.04 119 selenium headless-new,fullscreenEnabled: true,,fullscreen_iframe,https,sub.headers.websec.saarland,https,child_allow,headers.webappsec.eu,https://sub.headers.websec.saarland/_hp/tests/fullscreen-api-pp.sub.html?resp_type=parsing&browser_id=1&label=PP&first_id=37381&last_id=37381&scheme=https
2882440,chrome Ubuntu 22.04 119 selenium xvfb,message timeout,,fullscreen_iframe,https,sub.headers.websec.saarland,https,child_allow,headers.webappsec.eu,https://sub.headers.websec.saarland/_hp/tests/fullscreen-api-pp.sub.html?resp_type=parsing&browser_id=1&label=PP&first_id=37381&last_id=37381&scheme=https


fullscreen_iframe [36037, 37381, 37819, 38110, 38508]


,,count
outcome_str,browser,
fullscreenEnabled: true,chrome Ubuntu 22.04 119 selenium headless-new,15
message timeout,chrome Ubuntu 22.04 119 selenium xvfb,5


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
1822471,chrome Ubuntu 22.04 119 selenium headless-new,fullscreenEnabled: true,,fullscreen_iframe,https,sub.headers.websec.saarland,https,direct,headers.webappsec.eu,https://sub.headers.websec.saarland/_hp/tests/fullscreen-api-pp.sub.html?resp_type=parsing&browser_id=1&label=PP&first_id=37381&last_id=37381&scheme=https
2882439,chrome Ubuntu 22.04 119 selenium xvfb,message timeout,,fullscreen_iframe,https,sub.headers.websec.saarland,https,direct,headers.webappsec.eu,https://sub.headers.websec.saarland/_hp/tests/fullscreen-api-pp.sub.html?resp_type=parsing&browser_id=1&label=PP&first_id=37381&last_id=37381&scheme=https


## Browser differences!

In [43]:
df_stab = df

# Only keep one row for each test
# If the test had non-deterministic results, chose the first?
print("Original data entries", len(df))
df = df.drop_duplicates(subset=["test_name", "relation_info", "browser", "org_origin", "resp_origin", "response_id"])
print("Only one row for each test (per browser)", len(df))

# Remove browsers that currently do not have almost all results (safari + opera iOS, ucmobile android)
df = df.loc[~df["browser_id"].isin([4, 28, 30])]
print("Remove failed browsers", len(df))

Original data entries 3612487
Only one row for each test (per browser) 1349320
Remove failed browsers 1349320


In [46]:
# Most tests have exactly one outcome, some have two, one has three!
res = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
res = res["outcome_str"].agg("nunique")
display(res.value_counts().to_frame())

,count
outcome_str,
1,163050
2,5687
3,1


In [49]:
# oac_window.open has different results for most tests
# Other test groups have between 0.3 - 5% of tests with different outcomes
r = res.reset_index()
r["diff"] = r["outcome_str"] != 1
r.groupby(["test_name", "relation_info"])["diff"].agg(["count", "sum", "mean"]).sort_values("mean")

count   sum      mean
test_name                  relation_info                        
fetch_GET                  credentials     16118    46  0.002854
                           custom_headers  16118    61  0.003785
fetch_TEST                 custom_method   16118    61  0.003785
fetch_GET                  simple          16118    64  0.003971
perfAPI_img                direct           3202    14  0.004372
upgradeHSTS_subdomain      subdomain        6630    38  0.005732
subresourceloadingCOEP_img direct           5274    31  0.005878
accesswindow_direct        direct           5312    39  0.007342
framing_iframe             nested          17560   145  0.008257
subresourceloadingCORP_img direct           6714    56  0.008341
referrer_iframe            iframe           5594   101  0.018055
framing_iframe             direct          17560   365  0.020786
upgradeHSTS_direct         direct           6630   214  0.032278
fullscreen_iframe          direct           7760   277  0.035696
script_execution_iframe    direct           5452   216  0.039618
imgloading_iframe          direct           5420   230  0.042435
fullscreen_iframe          child_allow      7760   360  0.046392
oac_window.open            window.open      3398  3370  0.991760

In [50]:
# Select only test rows with more than one outcome
tests_with_mult_outcomes = r.loc[r["diff"]].set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
df_mult = df.loc[df[["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]].apply(tuple, axis=1).isin(tests_with_mult_outcomes.index)]

print(f"All test rows: {len(df)}, Rows with more than one outcome: {len(df_mult)}")

All test rows: 1349320, Tests with more than one outcome: 45504


In [52]:
# Display difference groups
for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    df_list = []
    for _, one_test in group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]):
        browsers = one_test.groupby("outcome_str")["browser"].unique().apply(list).apply(sorted).to_frame()
        new = True
        for df_b in df_list:
            if df_b.equals(browsers):
                new = False
        if new:
            df_list.append(browsers)
    print(len(df_list))
    for df_b in df_list:
        with pd.option_context("display.max_colwidth", 1000):
            display(df_b)
    #input("Continue!")

('accesswindow_direct', 'direct')
3


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb, firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb]"
"{'window.open.opener': 'object ""[object Window]""'}",[brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb]


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb]"
"{'window.open.opener': 'object ""[object Window]""'}","[firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb]"
"{'window.open.opener': 'object ""[object Window]""'}","[brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb]"


('fetch_GET', 'credentials')
6


...

,browser
outcome_str,
{'response.redirected': False},"[firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb]"
{'response.redirected': True},"[brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb]"


,browser
outcome_str,
{'response.redirected': False},"[brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb]"
{'response.redirected': True},"[firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb]"


('upgradeHSTS_subdomain', 'subdomain')
2


,browser
outcome_str,
{'response.redirected': False},"[firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb]"
{'response.redirected': True},"[brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb]"


,browser
outcome_str,
{'response.redirected': False},"[brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new, brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb, chrome Ubuntu 22.04 119 selenium headless-new, chrome Ubuntu 22.04 119 selenium xvfb, edge Ubuntu 22.04 119 selenium headless-new, edge Ubuntu 22.04 119 selenium xvfb]"
{'response.redirected': True},"[firefox Ubuntu 22.04 119 selenium headless, firefox Ubuntu 22.04 119 selenium xvfb]"


In [53]:
# Other approach to list the difference groups
def get_uniques(df):
    browsers = f"{df['name'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    
    unique_outcomes = {}
    group.groupby(["name"])["outcome_str"].value_counts(normalize=False).reset_index().groupby(["outcome_str", "count"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        display(d)
        print()

,['brave'],['firefox'],"['chrome', 'edge']"
{'window.open.opener': 'null'},43,46,54
"{'window.open.opener': 'object ""[object Window]""'}",35,32,24


,"['brave', 'chrome', 'edge']",['firefox']
"{'error': 'null', 'headers': 'content-length,'}",6,20
"{'error': 'null', 'headers': 'content-length,test,'}",38,40
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",48,32


,['firefox'],"['brave', 'chrome', 'edge']"
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",4,8
"{'error': 'null', 'headers': 'content-length,'}",28,8
"{'error': 'null', 'headers': 'content-length,test,'}",48,50
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",42,56


,"['brave', 'chrome', 'edge']",['firefox']
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,location,server,test,'}",2.0,NaN
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",8.0,4.0
"{'error': 'null', 'headers': 'content-length,'}",8.0,28.0
"{'error': 'null', 'headers': 'content-length,test,'}",50.0,52.0
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",60.0,24.0
"{'error': 'null', 'headers': 'cache-control,content-type,'}",NaN,20.0


,['firefox'],"['brave', 'chrome', 'edge']"
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",4,8
"{'error': 'null', 'headers': 'content-length,'}",28,8
"{'error': 'null', 'headers': 'content-length,test,'}",48,50
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",42,56


...

,['firefox'],"['brave', 'chrome', 'edge']"
message send,80,352
message timeout,352,80


,['firefox'],"['brave', 'chrome', 'edge']"
message timeout,12,36
"{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",6,10
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",44,16


,['firefox'],"['brave', 'chrome', 'edge']"
error,32,80
load,80,32


,"['brave', 'chrome', 'edge']",['firefox']
{'response.redirected': False},10,418
{'response.redirected': True},418,10


,"['brave', 'chrome', 'edge']",['firefox']
{'response.redirected': False},6,70
{'response.redirected': True},70,6


In [55]:
# Function to calculate absolute and percentage overlap
def calculate_overlap(df, col1, col2):
    df = df[[col1, col2]].dropna()
    df.columns.values[0] = 0
    df.columns.values[1] = 1
    absolute_overlap = (df[0] == df[1]).sum()
    unique_rows = len(df)
    diff_rows = unique_rows - absolute_overlap
    percentage_diff = (diff_rows / unique_rows) * 100
    return diff_rows, percentage_diff, unique_rows


def display_overlap(df, show_all=True, name="All"):
    print(name)
    sim_frame = df.set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])["outcome_str"].unstack().astype("str")
    sim_frame = sim_frame.reset_index().drop(columns=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
    # Create a dictionary to store results
    results = {}
    matrix = {}
    
    # Iterate through all pairs of columns
    for i, col1 in enumerate(sim_frame.columns):
        for col2 in sim_frame.columns:
            key = f"{col1}_vs_{col2}"
            results[key] = calculate_overlap(sim_frame, col1, col2)
            try:
                l = matrix[col1]
            except KeyError:
                l = {}
            # 0 for absolute count of differences, 1 for percentage
            l[col2] = results[key][0]
            matrix[col1] = l
            
    
    # Convert the results to a DataFrame for better visualization
    results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Absolute Diff', 'Percentage Diff', "Count"])
    
    # Display the results
    if show_all:
        display(results_df)
    
    # Display difference matrix
    with pd.option_context("display.max_columns", 28):
        if show_all:
            display(pd.DataFrame(matrix))
        else:
            display(pd.DataFrame(matrix).drop_duplicates().T)


display_overlap(df)

for name, group in df.groupby("test_name"):
    display_overlap(group, show_all=False, name=name)

All


,Absolute Diff,Percentage Diff,Count
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new_vs_brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,0,0.000000,168738
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new_vs_brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,11,0.006519,168738
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new_vs_chrome Ubuntu 22.04 119 selenium headless-new,70,0.041484,168738
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new_vs_chrome Ubuntu 22.04 119 selenium xvfb,82,0.048596,168738
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new_vs_edge Ubuntu 22.04 119 selenium headless-new,70,0.041484,168738
...,...,...,...
firefox Ubuntu 22.04 119 selenium xvfb_vs_chrome Ubuntu 22.04 119 selenium xvfb,5896,3.494174,168738
firefox Ubuntu 22.04 119 selenium xvfb_vs_edge Ubuntu 22.04 119 selenium headless-new,5886,3.488248,168738
firefox Ubuntu 22.04 119 selenium xvfb_vs_edge Ubuntu 22.04 119 selenium xvfb,5886,3.488248,168738
firefox Ubuntu 22.04 119 selenium xvfb_vs_firefox Ubuntu 22.04 119 selenium headless,0,0.000000,168738


,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,chrome Ubuntu 22.04 119 selenium headless-new,chrome Ubuntu 22.04 119 selenium xvfb,edge Ubuntu 22.04 119 selenium headless-new,edge Ubuntu 22.04 119 selenium xvfb,firefox Ubuntu 22.04 119 selenium headless,firefox Ubuntu 22.04 119 selenium xvfb
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,0,11,70,82,70,70,5817,5817
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,11,0,81,93,81,81,5828,5828
chrome Ubuntu 22.04 119 selenium headless-new,70,81,0,12,0,0,5886,5886
chrome Ubuntu 22.04 119 selenium xvfb,82,93,12,0,12,12,5896,5896
edge Ubuntu 22.04 119 selenium headless-new,70,81,0,12,0,0,5886,5886
edge Ubuntu 22.04 119 selenium xvfb,70,81,0,12,0,0,5886,5886
firefox Ubuntu 22.04 119 selenium headless,5817,5828,5886,5896,5886,5886,0,0
firefox Ubuntu 22.04 119 selenium xvfb,5817,5828,5886,5896,5886,5886,0,0


accesswindow_direct


,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,firefox Ubuntu 22.04 119 selenium headless
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,0,11,28
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,11,0,39
chrome Ubuntu 22.04 119 selenium headless-new,0,11,28
chrome Ubuntu 22.04 119 selenium xvfb,0,11,28
edge Ubuntu 22.04 119 selenium headless-new,0,11,28
edge Ubuntu 22.04 119 selenium xvfb,0,11,28
firefox Ubuntu 22.04 119 selenium headless,28,39,0
firefox Ubuntu 22.04 119 selenium xvfb,28,39,0


...

,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,firefox Ubuntu 22.04 119 selenium headless
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,0,56
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,0,56
chrome Ubuntu 22.04 119 selenium headless-new,0,56
chrome Ubuntu 22.04 119 selenium xvfb,0,56
edge Ubuntu 22.04 119 selenium headless-new,0,56
edge Ubuntu 22.04 119 selenium xvfb,0,56
firefox Ubuntu 22.04 119 selenium headless,56,0
firefox Ubuntu 22.04 119 selenium xvfb,56,0


upgradeHSTS_direct


,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,firefox Ubuntu 22.04 119 selenium headless
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,0,244
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,0,244
chrome Ubuntu 22.04 119 selenium headless-new,0,244
chrome Ubuntu 22.04 119 selenium xvfb,0,244
edge Ubuntu 22.04 119 selenium headless-new,0,244
edge Ubuntu 22.04 119 selenium xvfb,0,244
firefox Ubuntu 22.04 119 selenium headless,244,0
firefox Ubuntu 22.04 119 selenium xvfb,244,0


upgradeHSTS_subdomain


,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,firefox Ubuntu 22.04 119 selenium headless
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium headless-new,0,68
brave Ubuntu 22.04 1.60.114 (119.0.6045.124) selenium xvfb,0,68
chrome Ubuntu 22.04 119 selenium headless-new,0,68
chrome Ubuntu 22.04 119 selenium xvfb,0,68
edge Ubuntu 22.04 119 selenium headless-new,0,68
edge Ubuntu 22.04 119 selenium xvfb,0,68
firefox Ubuntu 22.04 119 selenium headless,68,0
firefox Ubuntu 22.04 119 selenium xvfb,68,0


In [56]:
# How many rows exist for tree creation
df[["test_name", "label", "relation_info"]].value_counts()

test_name                   label       relation_info 
framing_iframe              XFO         direct            73536
                                        nested            73536
fullscreen_iframe           PP          child_allow       62080
                                        direct            62080
subresourceloadingCORP_img  CORP        direct            53712
upgradeHSTS_direct          HSTS        direct            52908
upgradeHSTS_subdomain       HSTS        subdomain         52908
referrer_iframe             RP          iframe            44752
framing_iframe              CSP-FA      direct            43904
                                        nested            43904
script_execution_iframe     CSP-SCRIPT  direct            43616
imgloading_iframe           CSP-IMG     direct            43360
accesswindow_direct         COOP        direct            42496
subresourceloadingCOEP_img  COEP        direct            42192
fetch_TEST                  CORS-ACAO   custom_me

In [ ]:
df["raw_header"] = df["raw_header"].apply(bytes).apply(lambda x: json.loads(x.decode("utf-8"))).astype(str)

In [60]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()


# Limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])["outcome_str"].transform("nunique") != 1
tree_df = df.loc[condition]
print("Remove tests that are the same in all browsers", len(tree_df))
base_dir = f"trees/{datetime.today().strftime('%Y-%m-%dT%H:%M')}" 

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]
    group = group[["outcome_str", *pred_cols]]
    tree = make_tree(group, pred_cols, group_name, base_dir)
    # print(tree)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,23 days 0 hours 22 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,2 months and 25 days
H2O_cluster_name:,H2O_from_python_ubuntu_jovqdg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Remove tests that are the same in all browsers 45504
('accesswindow_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,23 days 0 hours 23 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,2 months and 25 days
H2O_cluster_name:,H2O_from_python_ubuntu_jovqdg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('accesswindow_direct', 'direct'), datapoints: 312
('fetch_GET', 'credentials')
Connecting to H2O server at http://localhost:54321 ... successful.


...

Create tree: ('subresourceloadingCORP_img', 'direct'), datapoints: 448
('upgradeHSTS_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,23 days 0 hours 23 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,2 months and 25 days
H2O_cluster_name:,H2O_from_python_ubuntu_jovqdg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_direct', 'direct'), datapoints: 1712
('upgradeHSTS_subdomain', 'subdomain')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,23 days 0 hours 23 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,2 months and 25 days
H2O_cluster_name:,H2O_from_python_ubuntu_jovqdg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_subdomain', 'subdomain'), datapoints: 304
